In [ ]:
# -*- coding: utf-8 -*-
"""scc 461 py Decision tree.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1TP0t-pgIZHf4D4KUvhtD6T5ZXhMAtmrQ

## Basics
"""

from sklearn import datasets
from sklearn import model_selection
import pandas as pd
import warnings
#from sklearn.tree import DecisionTreeClassifier
import time 
import matplotlib.pyplot as plt
import graphviz
from enum import Enum

warnings.filterwarnings('ignore')

class NodeTypes(Enum):
  LEAF = "Leaf"
  PARENT = "Parent"
  ROOT = "Root"

wineDataset = datasets.load_wine()

featureSet, targetSet = wineDataset.data, wineDataset.target

dfFeatures = pd.DataFrame(featureSet, columns = wineDataset.feature_names)
dfTarget = pd.DataFrame(targetSet, columns = ["Class"])

"""## EDA"""

dfFeatures.head()

dfFeatures.info()

print(len(dfTarget['Class'].value_counts()))

"""## Feature Engineering

### Handle Categorical variables

encoding

Ref: https://www.kaggle.com/prashant111/decision-tree-classifier-tutorial#12.-Feature-Engineering-
"""

#import category_encoders as ce

"""## Train and Test datasets"""

import sklearn as skl
from sklearn import tree

trainFeatureSet, testFeatureSet, trainTargetSet, testTargetSet = model_selection.train_test_split(dfFeatures, dfTarget,
                                                                                                      train_size = 0.7, test_size = 0.3,
                                                                                                      shuffle = True)

"""## Helper methods"""

def getEvaluationMetrics(predictedTargetSet): 
  accuracy = skl.metrics.accuracy_score(testTargetSet, predictedTargetSet)
  precision = skl.metrics.precision_score(testTargetSet, predictedTargetSet, average='micro')
  recall = skl.metrics.recall_score(testTargetSet, predictedTargetSet, average='micro')
  f1score = skl.metrics.f1_score(testTargetSet, predictedTargetSet, average='micro')
  classificationReport = skl.metrics.classification_report(testTargetSet, predictedTargetSet)
  return {
      accuracy: accuracy,
      precision: precision,
      recall: recall,
      f1score: f1score,
      classificationReport: classificationReport
  }

def printEvaluationMetrics(predictedTargetSet):
  evalMetrics = getEvaluationMetrics(predictedTargetSet)
  print (evalMetrics)
  print(f"Accuracy score of model: {evalMetrics.accuracy}")
  print(f"Precision score of model: {evalMetrics.precision}")
  print(f"Recall score of model: {evalMetrics.recall}")
  print(f"F1 of model: {evalMetrics.f1score}")
  print(f"Classification report of model: {evalMetrics.classificationReport}")

def getTimeTaken(startTime, endTime):
  return round((endTime - startTime) * 100, 5)

def getTrainingPerformanceMetrics():
  return {
      timeTaken: timeTaken
  }

def getAverage(a, b):
    #lambda x, y: (x[colIndex]+y[colIndex])/2
    return (a+b)/2
"""## Decision Tree

### Helper methods for Decision tree
"""

#@title
#Calculation of gini value
def giniValue(localtrainingSet) -> float:
  giniValue = sumOfProbabilities = 0 
  #print(localtrainingSet)
  classes = localtrainingSet["Class"].unique()

  if len(classes)>2:
    for i in range(classes):
      localTreeClass = filter(localtrainingSet, lambda row: row["Class"] == classes[i]) #Filter for each class
      sumOfProbabilities+= (len(localTreeClass) / len(tree)) ** 2 
    giniValue = 1 - sumOfProbabilities
  return giniValue #return the Gini Index value for this split
  #return False

#find the best partition that minimizes the Gini index:
def getGiniIndexValue( splitPoints: list, colIndex: int, dataset) -> tuple(): #returns tuple with (Gini index value, the split value)
  localLeftTree = localRightTree = [] = giniIndexes = []
  colName = dataset.columns[colIndex]
  for midVal in splitPoints:
      localLeftTree = dataset.loc[dataset[colName] <= midVal] #list(filter(lambda x: x <= midVal, dataset))
      localRightTree = dataset.loc[dataset[colName] > midVal] #list(filter(lambda x: x > midVal, dataset))
      
      D1 = len(localLeftTree); D2 = len(localRightTree); D = len(dataset)

      if D >0:        #weighted average of gini impurities
          localGiniImpurity = ( (D1/ D) * (giniValue(localLeftTree)) + (D2/ D) * (giniValue(localRightTree)))
          giniIndexes.append((localGiniImpurity, midVal))

  if len(giniIndexes) > 0 :
      return sorted(giniIndexes)[0]# returns the tuple with the smallest gini index value
  
  return False #Split could not be determined

#@title
# from sklearn.tree import _tree

# def tree_to_code(tree, feature_names):
#     tree_ = tree.tree_
#     feature_name = [
#         feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
#         for i in tree_.feature
#     ]
#     print "def tree({}):".format(", ".join(feature_names))

#     def recurse(node, depth):
#         indent = "  " * depth
#         if tree_.feature[node] != _tree.TREE_UNDEFINED:
#             name = feature_name[node]
#             threshold = tree_.threshold[node]
#             print "{}if {} <= {}:".format(indent, name, threshold)
#             recurse(tree_.children_left[node], depth + 1)
#             print "{}else:  # if {} > {}".format(indent, name, threshold)
#             recurse(tree_.children_right[node], depth + 1)
#         else:
#             print "{}return {}".format(indent, tree_.value[node])

#     recurse(0, 1)

#@title
def prechecksOnDataSet(dataSet):

  if len(dataSet['Class'].value_counts()) <2: #Atleast 2 classes in the target
    return False
  return True

"""### Helper Classes"""

class Node:  #class definition
    def __init__(self, feature, splitValue, testCriteria, left=None, right=None):  
        #Question - column, value
        self.feature = feature
        self.testCriteria = testCriteria
        self.splitValue = splitValue 
        self.nodeType = None # (Root, parent, child) 
        self.left = left
        self.right = right

class LocalNode(Node): 
    def __init__(self, gini, feature, splitValue, testCriteria):
      Node.__init__(self, feature, splitValue, testCriteria)  
      self.gini = gini

class Tree:
  def __init__(self, rootNode: Node = None):
    self.root = rootNode

trainFeatureSet["alcohol"].dtype.name

"""### Decision Tree"""

#@title
class DecisionTree:     #Declaring DecisionTree
  #Constructor
  def __init__(self, criterion: str = "gini", maxDepth: int = None) -> None:
    self.criterion = criterion
    self.maxDepth = maxDepth
    self.dfFeatures = 0
    self.dfTarget = 0
    self.tree = Tree() #Set of rules for our model

  def modelFit(self, featureSet, targetSet):
    self.dfFeature = featureSet
    self.dfTarget = targetSet

    if (prechecksOnDataSet(targetSet)):
      self._train(featureSet.join(targetSet))

  def _train(self, trainingSet):
    self.Tree = Tree(self._findBestSplit(trainingSet))

  def _findBestSplit(self, trainingSet) -> tuple():
    gini = split = 0
    leftTree = rightTree = []
    listOfAllColsAndGini = []

    for colIndex in range(len(trainingSet.columns)-1):
      splitPoints = []
      sortedData = trainingSet.copy()
      columnData = trainingSet.iloc[:,colIndex]
      columnType = trainingSet.iloc[:,colIndex].dtype.name
      columnName = trainingSet.columns[colIndex]
      if (columnType == "float64" or  columnType == "float64" ): # continuous integer features, sort it in ascending order 
        columnData = sortedData.sort_values(by=columnName) #.iloc[:,colIndex]
        
        if (columnData.shape[0] < 200):
            for i in range(columnData.shape[0] -1):
                splitPoints.append( (columnData.iloc[i, colIndex] + columnData.iloc[i+1, colIndex])/2)
          #splitPoints = list(map(getAverage, columnData[:-1], columnData[1:])) #midpoints are the splitpoints 
        #else:
          #splitPoints = list(filter( lambda m: m not in list(map(lambda y: y[0], sortedData)), midpoints)) #different splitpoints to optimize

        #columnData = sortedData
        if len(splitPoints) == 0:
          continue;
      elif columnType == "category":
        continue;
      else:
        continue

      #Get Gini Index value and the split point value
      giniIndexVal= getGiniIndexValue(splitPoints, colIndex, columnData)
      if giniIndexVal == False:
          continue;

      gini, split = giniIndexVal
      gini = round(gini, 5)   
      node = LocalNode(gini = gini, feature= dfFeatures.columns[colIndex], splitValue= split, testCriteria="<=")
      listOfAllColsAndGini.append(node)

    #We use the split point value calculated to get the left and right branches of the decision tree
    bestNode = sorted(listOfAllColsAndGini)[0]
    parentNode= Node(bestNode["feature"], )
    leftTree = list(filter( lambda x:  x[colIndex] <= split, sortedData))
    rightTree = list(filter( lambda x:  x[colIndex] > split, sortedData))
  
    return (gini, split, leftTree, rightTree)

dTreeOrg = DecisionTree()

dTreeOrg.modelFit(dfFeatures, dfTarget)

# def _giniValue(self, tree: DataFrame) -> float:
  #   giniValue =  sumOfProbabilities = 0 
  #   classes = self.dfTarget.Class.unique()

  #   if len(tree)>0:
  #     for i in range(classes):
  #       localTreeClass = filter(tree) #Filter for each class
  #       sumOfProbabilities+= (len(localTreeClass) / len(tree)) ** 2 
  #     giniValue = 1 - sumOfProbabilities
  #     return giniValue #return the Gini Index value for this split
  #   return False

# =============================================================================
# """## sklearn Decision tree
# 
# """
# 
# dTreeSkl = tree.DecisionTreeClassifier(criterion='gini', random_state=0) #max_depth=3,
# 
# """### Train model"""
# 
# dTreeSklStartTime = time.time()
# dTreeSkl.fit(trainFeatureSet, trainTargetSet)
# dTreeSklEndTime = time.time()
# dTreeSklTimeTaken = getTimeTaken(dTreeSklStartTime, dTreeSklEndTime) 
# 
# print(f"Time taken to train Decision Tree:", dTreeSklTimeTaken)
# 
# 
# 
# """### Test model"""
# 
# predictedValues = dTreeSkl.predict(testFeatureSet)
# print(f"Evaluation Metrics:", getEvaluationMetrics(predictedTargetSet = predictedValues))
# 
# #trainTargetSet
# skl.metrics.accuracy_score(dTreeSkl.predict(trainFeatureSet), trainTargetSet)
# 
# """### Visualizing"""
# 
# #@title
# 
# dot_data = skl.tree.export_graphviz(dTreeSkl, out_file=None,  
#                 filled=True, rounded=False,
#                 special_characters=True,
#                 feature_names = wineDataset.feature_names,
#                 class_names = wineDataset.target_names)
# graph = graphviz.Source(dot_data)  
# graph
# =============================================================================

"""## Chefboost"""

# pip install chefboost

# from chefboost import Chefboost as chef

# df = dfFeatures.join(dfTarget)
# config = {'algorithm': 'CART'} #Algo selection 
# #Other options for classification- CART, C4.5, CHAID, ID3,
# df['Class'] = df['Class'].astype(object) #requires 'Decision'/ Label column to be object dtype

# model2 = chef.fit(df, config = config, target_label = 'Class')

# prediction = chef.predict(model2, param = testFeatureSet.to_numpy())

# testFeatureSet.to_numpy()[0,12] <=746.8932584269663

# """# Rough notes (Ignore)"""

# trainFeatureSet.corr()